#Реализация LSTM (продолжение)
В этом уроке мы продолжим знакомство с LSTM и рассмотрим подробнее флаг `return_sequences`

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.X

Переключаемся на версию 2.0 (работает только в Colab)

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


### Загрузка библиотек
TensorFlow должен иметь как минимум версию 2.0

In [ ]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.0.0


### Входной тензор
Подготовим входной тензор для тестов аналогично тому, как мы это делали в предыдущем уроке.

Размерности тензора `x`: (батч, длина цепочки, размер эмбеддинга)

In [ ]:
BATCH_SIZE = 2
SEQ_LEN = 100 # Длина последовательности
EMB_SIZE = 16 # Размер векторного представления (эмбеддинга)

x = np.random.rand(BATCH_SIZE, SEQ_LEN, EMB_SIZE).astype(np.float32)
print(x.shape)

(2, 100, 16)


### Флаг `return_sequences`

В рекуррентных слоях, как правило, есть флаг `return_sequences`. Ранее мы использовали `return_sequences=True`

Если же установить флаг `return_sequences=False`, то рекуррентный слой будет выдавать лишь **последний** вектор `h`, а не всю цепочку векторов `h`.

In [ ]:
H_SIZE = 32
lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=False, recurrent_activation='sigmoid')

Запускаем инференс (прямое распространение) для созданного слоя и проверяем размерность выходного тензора.

Она должна была получиться `(BATCH_SIZE, H_SIZE)`

In [ ]:
y = lstm(x)
print(y.shape)

(2, 32)


**[Задание 1]** Модернизируйте класс `LSTM` из предыдущего урока, добавив в него (в конструктор) флаг `return_sequences`, чтобы поддерживались обе опции (True/False).

**[Задание 2]** Сравните результаты работы нового класса `LSTM` (с параметром `return_sequences=False`) с исходной моделью на Keras так же, как мы это делали раньше (разница должна получиться равной нулю или очень маленькой)

In [ ]:
# LSTM ячейка из предыдущего урока
class LSTMCell(tf.keras.Model):
    def __init__(self, h_size):
        super().__init__()
        self.h_size = h_size
        
        # input gate
        self.fcXI = tf.keras.layers.Dense(self.h_size)
        self.fcHI = tf.keras.layers.Dense(self.h_size, use_bias=False)
        
        # forget gate
        self.fcXF = tf.keras.layers.Dense(self.h_size)
        self.fcHF = tf.keras.layers.Dense(self.h_size, use_bias=False)
        
        # создание нового кандидата CH
        self.fcXC = tf.keras.layers.Dense(self.h_size)
        self.fcHC = tf.keras.layers.Dense(self.h_size, use_bias=False)
        
        # output gate
        self.fcXO = tf.keras.layers.Dense(self.h_size)
        self.fcHO = tf.keras.layers.Dense(self.h_size, use_bias=False)
        
    def call(self, x, h, c):
        i = tf.nn.sigmoid(self.fcXI(x) + self.fcHI(h))
        f = tf.nn.sigmoid(self.fcXF(x) + self.fcHF(h))
        o = tf.nn.sigmoid(self.fcXO(x) + self.fcHO(h))
        ch = tf.nn.tanh(self.fcXC(x) + self.fcHC(h))
        c = f*c + i*ch
        h = o*tf.nn.tanh(c)
        return h, c

In [ ]:
# Теперь тут добавлен флаг return_sequences
class LSTM(tf.keras.Model):
    def __init__(self, h_size, return_sequences=True):
        super().__init__()
        self.h_size = h_size
        self.return_sequences=return_sequences
        self.lstm_cell = LSTMCell(h_size)

    def call(self, x_all):
        batch, length, emb_size = x.shape           
        h = tf.zeros((batch, self.h_size))
        c = tf.zeros((batch, self.h_size))
        
        if self.return_sequences:
            h_all = []
        
        for i in range(length):
            h, c = self.lstm_cell(x_all[:, i, :], h, c)
            if self.return_sequences:
                h_all.append(h)
        
        if self.return_sequences:
            h_all = tf.transpose(tf.stack(h_all), [1, 0, 2])
            return h_all
        else:
            return h

lstm_my = LSTM(H_SIZE, False)

In [ ]:
y = lstm_my(x)
print(y.shape)

(2, 32)


In [ ]:
# Перед тем, как что-то присваивать в параметры модели, нужно чтобы они создались.
# Для этого можно вызвать либо инференс с каким-то входом, либо model.build(...)

lstm_my.lstm_cell.fcXI.kernel = lstm.weights[0][:, 0:32]
lstm_my.lstm_cell.fcXF.kernel = lstm.weights[0][:, 32:64]
lstm_my.lstm_cell.fcXC.kernel = lstm.weights[0][:, 64:96]
lstm_my.lstm_cell.fcXO.kernel = lstm.weights[0][:, 96:128]

lstm_my.lstm_cell.fcHI.kernel = lstm.weights[1][:, 0:32]
lstm_my.lstm_cell.fcHF.kernel = lstm.weights[1][:, 32:64]
lstm_my.lstm_cell.fcHC.kernel = lstm.weights[1][:, 64:96]
lstm_my.lstm_cell.fcHO.kernel = lstm.weights[1][:, 96:128]

lstm_my.lstm_cell.fcXI.bias = lstm.weights[2][0:32]
lstm_my.lstm_cell.fcXF.bias = lstm.weights[2][32:64]
lstm_my.lstm_cell.fcXC.bias = lstm.weights[2][64:96]
lstm_my.lstm_cell.fcXO.bias = lstm.weights[2][96:128]

y = lstm(x)
y_my = lstm_my(x)

print(np.max(np.abs(y.numpy() - y_my.numpy())))

5.9604645e-08
